### Parallel workflow with LLM

Start(essay) -> 1) Clarity of text(produce Sumary+score)   -> Final eval(final sumary+score)
             -> 2) Depth of Analysis(produce Sumary+score) ->
             -> 3) Language(produce Sumary+score)          ->

In [46]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field
import operator

In [47]:
load_dotenv()
model = ChatGroq(model="meta-llama/llama-4-maverick-17b-128e-instruct", temperature=0)

In [48]:
# Create class for getting structured output
class EvaluationSchema(BaseModel):
    feedback : str = Field(description="Detailed feedback for the essay")
    score: int = Field(description="Give the score out of 10", ge=0, le=10)

In [49]:
# Creating structured output model
structured_model = model.with_structured_output(EvaluationSchema)

In [50]:
essay = """The Role of Technology in Modern Education

In the twenty-first century, technology has become one of the most powerful tools shaping every aspect of human life. Among its many areas of influence, education has seen a dramatic transformation. From interactive classrooms to online learning platforms, technology has redefined how knowledge is delivered, accessed, and retained. While some critics argue that it reduces personal interaction and creates dependency, the benefits of technology in education outweigh its drawbacks by fostering accessibility, engagement, and lifelong learning.

One of the most significant contributions of technology to education is increased accessibility. In the past, students in remote or underdeveloped regions often lacked access to quality teachers and resources. Today, with the internet and digital devices, learners across the globe can access lectures from world-class universities, join virtual classrooms, and download educational materials. This democratization of education ensures that learning is no longer limited by geography or financial constraints.

Beyond accessibility, technology also enhances student engagement. Traditional methods of teaching—based mainly on lectures and textbooks—often fail to capture students’ attention. Interactive whiteboards, educational apps, and multimedia resources make lessons more dynamic and relatable. For example, virtual reality can bring history to life by allowing students to “visit” ancient civilizations, while simulations in science enable them to experiment in safe, controlled environments. These tools not only improve comprehension but also encourage curiosity and creativity.

Furthermore, technology fosters lifelong learning, an essential skill in a rapidly changing world. With industries constantly evolving, individuals must continuously update their knowledge and skills. Online courses, webinars, and digital certifications allow people to learn at their own pace, even while managing careers and family responsibilities. This flexibility helps create a culture of continuous growth, preparing individuals for the challenges of the modern workforce.

However, it is important to acknowledge the potential drawbacks. Overreliance on technology can limit face-to-face interactions, which are crucial for developing communication and social skills. Moreover, unequal access to devices and the internet may deepen the gap between privileged and underprivileged learners. To address these issues, governments and institutions must balance digital education with traditional methods while ensuring equitable access to resources.

In conclusion, technology has revolutionized modern education by breaking barriers of distance, enhancing engagement, and promoting lifelong learning. While challenges remain, the responsible integration of digital tools promises to make education more inclusive, effective, and adaptable to the future. Ultimately, technology should not replace teachers but empower them, creating a learning environment that combines human guidance with the limitless possibilities of innovation.


"""

In [51]:
# 
prompt = f"Evaluate the language quality of the essay and provide a feedback and assign a score out of 10 based on {essay}"
structured_model.invoke(prompt)

InternalServerError: Error code: 500 - {'error': {'message': 'Internal Server Error', 'type': 'internal_server_error'}}

In [ ]:
# Create a State
class UPSCState(TypedDict):
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add]

NameError: name 'Annotated' is not defined